In [1]:
import os,gc, pandas as pd
pd.set_option('display.max_columns', 500)
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/train_data/test.parquet')
mn = df.ts.min()
#df['hour'] = df.ts % (60*60*24)
df['day'] = (df.ts-mn)  // (60*60*24) #% (60*60*24*7)
print( df.shape )
df.head()

(7683577, 5)


,session,aid,ts,type,day
0,11098528,11830,1661119200,0,0
1,11098529,1105029,1661119200,0,0
2,11098530,264500,1661119200,0,0
3,11098530,264500,1661119288,0,0
4,11098530,409236,1661119369,0,0


In [3]:
AIDS = df.aid.unique().values
DAYS = df.day.unique().values

In [4]:
DAYS

array([0, 1, 2, 3, 4, 5, 6])

In [5]:
tmp = df.groupby(['aid','day']).agg({'aid':'count','session':'count'})
tmp.columns = ['item','user']

In [6]:
tmp.head()

,,item,user
aid,day,,
914103,1,2,2
1421276,2,43,43
1197590,4,1,1
187947,3,9,9
1718694,0,6,6


In [7]:
import cupy
dff = cudf.DataFrame(columns=['aid','day'])
dff['aid'] = cupy.repeat(AIDS,len(DAYS)).astype('int32')
dff['day'] = cupy.tile(DAYS,len(AIDS)).astype('int8')
dff = dff.set_index( ['aid','day'] )

In [8]:
dff = dff.merge(tmp, left_index=True, right_index=True, how='left').fillna(0).reset_index()

In [9]:
dff.head()

,aid,day,item,user
0,6935,3,1,1
1,6936,0,1,1
2,6940,0,1,1
3,6935,4,0,0
4,6935,5,0,0


In [10]:
item_features3 = dff.groupby('aid').agg({'item':'std','user':'std'}).fillna(-1)
item_features3.columns = ['item_day_std','user_day_std']

f32 = ['item_day_std','user_day_std']
for c in f32: item_features3[c] = item_features3[c].astype('float32')

In [11]:
del dff
gc.collect()

36

In [12]:
df.head()

,session,aid,ts,type,day
0,11098528,11830,1661119200,0,0
1,11098529,1105029,1661119200,0,0
2,11098530,264500,1661119200,0,0
3,11098530,264500,1661119288,0,0
4,11098530,409236,1661119369,0,0


In [13]:
item_features = df.groupby(['aid','day']).agg({'type':'mean','aid':'count','session':'nunique'})
item_features.columns = ['buy_ratio','count_item','count_user']

f32 = ['buy_ratio']
for c in f32: item_features[c] = item_features[c].astype('float32')
    
i32 = ['count_item','count_user']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [14]:
item_features.head()

buy_ratio  count_item  count_user
aid day                                   
0   2          0.0           2           1
    4          0.0           1           1
    5          0.0           1           1
    6          0.0           1           1
2   0          0.0           2           1

In [15]:
dff = cudf.DataFrame(columns=['aid','day'])
dff['aid'] = cupy.repeat(AIDS,len(DAYS)).astype('int32')
dff['day'] = cupy.tile(DAYS,len(AIDS)).astype('int8')
dff = dff.set_index( ['aid','day'] )

In [16]:
dff = dff.merge(item_features, left_index=True, right_index=True, how='left').fillna(-1).reset_index()
dff = dff.sort_values(['aid','day'])

In [17]:
dff2 = cudf.DataFrame(index=AIDS)
for c in ['buy_ratio','count_item','count_user']:
    v = dff[c].values.reshape((-1,7))
    for k in range(7):
        n = f'{c}_{k}'
        dff2[n] = v[:,k] 

In [18]:
del dff
gc.collect()

0

In [19]:
dff2.head()

,buy_ratio_0,buy_ratio_1,buy_ratio_2,buy_ratio_3,buy_ratio_4,buy_ratio_5,buy_ratio_6,count_item_0,count_item_1,count_item_2,count_item_3,count_item_4,count_item_5,count_item_6,count_user_0,count_user_1,count_user_2,count_user_3,count_user_4,count_user_5,count_user_6
0,-1.000000,-1.0,0.000000,-1.000000,0.000000,0.000000,0.000000,-1,-1,2,-1,1,1,1,-1,-1,1,-1,1,1,1
2,0.000000,0.0,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,2,1,-1,-1,-1,1,-1,1,1,-1,-1,-1,1,-1
3,0.083333,0.0,-1.000000,0.236842,0.108696,0.047619,0.090909,60,18,-1,38,46,63,99,37,15,-1,16,25,39,54
4,-1.000000,-1.0,0.000000,0.000000,-1.000000,0.000000,0.000000,-1,-1,1,2,-1,2,3,-1,-1,1,2,-1,2,1
11,-1.000000,-1.0,0.333333,-1.000000,-1.000000,-1.000000,0.000000,-1,-1,3,-1,-1,-1,1,-1,-1,1,-1,-1,-1,1


In [20]:
dff2.dtypes

buy_ratio_0     float32
buy_ratio_1     float32
buy_ratio_2     float32
buy_ratio_3     float32
buy_ratio_4     float32
buy_ratio_5     float32
buy_ratio_6     float32
count_item_0      int32
count_item_1      int32
count_item_2      int32
count_item_3      int32
count_item_4      int32
count_item_5      int32
count_item_6      int32
count_user_0      int32
count_user_1      int32
count_user_2      int32
count_user_3      int32
count_user_4      int32
count_user_5      int32
count_user_6      int32
dtype: object

In [21]:
item_features = cudf.concat([dff2,item_features3],axis=1)

In [22]:
item_features.head()

,buy_ratio_0,buy_ratio_1,buy_ratio_2,buy_ratio_3,buy_ratio_4,buy_ratio_5,buy_ratio_6,count_item_0,count_item_1,count_item_2,count_item_3,count_item_4,count_item_5,count_item_6,count_user_0,count_user_1,count_user_2,count_user_3,count_user_4,count_user_5,count_user_6,item_day_std,user_day_std
0,-1.000000,-1.0,0.000000,-1.000000,0.000000,0.000000,0.000000,-1,-1,2,-1,1,1,1,-1,-1,1,-1,1,1,1,0.755929,0.755929
2,0.000000,0.0,-1.000000,-1.000000,-1.000000,0.000000,-1.000000,2,1,-1,-1,-1,1,-1,1,1,-1,-1,-1,1,-1,0.786796,0.786796
3,0.083333,0.0,-1.000000,0.236842,0.108696,0.047619,0.090909,60,18,-1,38,46,63,99,37,15,-1,16,25,39,54,32.294037,32.294037
4,-1.000000,-1.0,0.000000,0.000000,-1.000000,0.000000,0.000000,-1,-1,1,2,-1,2,3,-1,-1,1,2,-1,2,1,1.214986,1.214986
11,-1.000000,-1.0,0.333333,-1.000000,-1.000000,-1.000000,0.000000,-1,-1,3,-1,-1,-1,1,-1,-1,1,-1,-1,-1,1,1.133893,1.133893


In [23]:
item_features.columns = [f'{c}_6' for c in item_features.columns]

In [24]:
item_features.dtypes

buy_ratio_0_6     float32
buy_ratio_1_6     float32
buy_ratio_2_6     float32
buy_ratio_3_6     float32
buy_ratio_4_6     float32
buy_ratio_5_6     float32
buy_ratio_6_6     float32
count_item_0_6      int32
count_item_1_6      int32
count_item_2_6      int32
count_item_3_6      int32
count_item_4_6      int32
count_item_5_6      int32
count_item_6_6      int32
count_user_0_6      int32
count_user_1_6      int32
count_user_2_6      int32
count_user_3_6      int32
count_user_4_6      int32
count_user_5_6      int32
count_user_6_6      int32
item_day_std_6    float32
user_day_std_6    float32
dtype: object

In [25]:
item_features.to_parquet('../../data/item_user_features/item13.pqt')